In [5]:
import numpy as np

def perturb_with_gaussian_noise(coordinates, std_x, std_y, std_z):
    
    # Ensure coordinates is a tuple or list with three elements
    if not isinstance(coordinates, (tuple, list)) or len(coordinates) != 3:
        raise ValueError("Coordinates should be a tuple or list with three elements (x, y, z).")

    # Unpack the input coordinates
    x, y, z = coordinates

    # Generate random perturbations from Gaussian distributions
    perturb_x = np.random.normal(loc=0, scale=std_x)
    perturb_y = np.random.normal(loc=0, scale=std_y)
    perturb_z = np.random.normal(loc=0, scale=std_z)

    # Apply perturbations to the coordinates
    x_perturbed = x + perturb_x
    y_perturbed = y + perturb_y
    z_perturbed = z + perturb_z

    # Return perturbed coordinates as a tuple
    return (x_perturbed, y_perturbed, z_perturbed)

In [6]:
coordinates = (123, 120, 20)
std_dev_x = 2
std_dev_y = 2
std_dev_z = 2
perturbed_coordinates = perturb_with_gaussian_noise(coordinates, std_dev_x, std_dev_y, std_dev_z)
print("Original Coordinates:", coordinates)
print("Perturbed Coordinates:", perturbed_coordinates)

Original Coordinates: (123, 120, 20)
Perturbed Coordinates: (118.83251286610702, 123.56935119528133, 19.181448478507754)


In [7]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def iceball_coverage_1(nrrd_file, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(sp)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate iceball coverage
    image_data = vtk_to_numpy(image.GetPointData().GetScalars())
    logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
    image_coverage = numpy.sum(image_data)  # Total coverage of the original image
    iceball_coverage = numpy.sum(logic_data)  # Coverage of the iceball within the image
    
    return iceball_coverage / image_coverage

In [30]:
from vtk.util.numpy_support import vtk_to_numpy
'''
Uses SD to find needle coordinates, finds tumor coverage, if tumor not entirely covered add a counter

Arguments: 
    - nrrd file
    - standard deviation in x, y, z directions
    - iceball center, tuple or array with 3 values
    - iceball radii, tuple or array with 3 values

Return: counter / simulation #
1. Loop through # times
2. Perturbed Coordinates
3. Tumor Ratio
4. If below 1.0
    Add to a counter
else keep on looping

Return percentage of tumor coverage

'''

def perturbed_tumor_percentage(nrrd, simulate, sd_x, sd_y, sd_z, center, radii):
    counter = 0
    for i in range (simulate):
        center_SD = center
        center_SD = perturb_with_gaussian_noise(center, sd_x, sd_y, sd_z)
        
        ratio = iceball_coverage_1(nrrd, center_SD, radii)
        print(ratio)
        if ratio < 1.0:
            counter += 1
    print (counter)
    print(f"Tumor Coverage Percentage: {counter / simulate * 100:.2f}%")

In [33]:
nrrd = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [123, 120, 20]
radii = [25.0, 25.0, 25.0]
simulate = 500
perturbed_tumor_percentage(nrrd, simulate, 2.0, 2.0, 2.0, center, radii)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5870786516853933
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7719101123595505
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8601123595505618
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7831460674157303
1.0
1.0
0.5988764044943821
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9691011235955056
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8129213483146067
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0